## 1.Import library

In [6]:
import sys
project_path = r"D:/DaiHoc/machinelearning/TLCN/DoAnTotNghiep_chat_bot/"
sys.path.append(project_path)
from source.function.utils_result import RAG
from source.search.utils_search import Qdrant_Utils
from source.rerank.utils_rerank import Rerank_Utils  
from source.model.embedding_model import Sentences_Transformer_Embedding
from source.model.extract_model import Bert_Extract
from source.model.generate_model import Gemini
from source.model.rerank_model import Cohere
from source.data.vectordb.qdrant import Qdrant_Vector
from source.core.config import Settings
from source.generate.generate import Gemini_Generate
from source.extract.utils_extract import Extract_Information
from source.schema.chatbot_querry import ChatbotQuery

setting=Settings()
gemini=Gemini(setting)
cohere=Cohere(setting)
bert=Bert_Extract(setting)
sentences_transformer_embedding=Sentences_Transformer_Embedding(setting)
qdrant=Qdrant_Vector(setting,sentences_transformer_embedding)
rerank_Utils=Rerank_Utils(cohere)
extract_Utils= Extract_Information(bert)
generate_Utils=Gemini_Generate(gemini,setting)
qdrant_Utils=Qdrant_Utils(qdrant, generate_Utils)
rag=RAG(generate_Utils,extract_Utils,qdrant_Utils,rerank_Utils,setting,sentences_transformer_embedding)
import pandas as pd
from tqdm import tqdm
import google.generativeai as genai 
from sklearn.metrics.pairwise import cosine_similarity
import re
import json

## 2. Load file dataset Generate Query

In [2]:
# Đọc file dataset_query_entities
file_path = "../data_danhgia_module_router_generate/dataset_Query_Entities.xlsx"

df_query_entities = pd.read_excel(file_path, engine="openpyxl")

# Hiển thị 5 dòng đầu tiên
print(df_query_entities.head(5))

   STT                                              query  \
0    1  Trong Thông tư số 57/2015/TT-BYT do Bộ Y tế ba...   
1    2  Trong lĩnh vực Thể thao - Y tế, Thông tư số 57...   
2    3  Trong Thông tư số 57/2015/TT-BYT của Bộ Y tế, ...   
3    4  Trong văn bản pháp luật liên quan đến quyền dâ...   
4    5  Văn bản do Bộ Y tế ban hành ngày 30/12/2015 có...   

                                      entities_label  
0  {"Loai-Van-Ban": "Thông tư", "Noi-Ban-Hanh": "...  
1  {"LinhVuc-Nganh": "Thể thao - Y tế", "Loai-Van...  
2  {"Loai-Van-Ban": "Thông tư", "Noi-Ban-Hanh": "...  
3  {"Noi-Ban-Hanh": "Bộ Y tế", "LinhVuc-Nganh": "...  
4  {"Noi-Ban-Hanh": "Bộ Y tế", "Ngay-Ban-Hanh": "...  


> các key duy nhất

In [3]:
import json

all_keys = set()

for val in df_query_entities['entities_label'].unique():
    try:
        entity_dict = json.loads(val)
        all_keys.update(entity_dict.keys())
    except Exception as e:
        print(f"Lỗi khi xử lý chuỗi: {val} - {e}")

print("Các key duy nhất trong entities_label:")
for key in sorted(all_keys):
    print(f"- {key}")


Các key duy nhất trong entities_label:
- LinhVuc-Nganh
- Loai-Van-Ban
- Ngay-Ban-Hanh
- Noi-Ban-Hanh
- So-Hieu


### 3.CALL API Gemini

In [7]:
setting=Settings()
gemini=Gemini(setting)
gemini_func=Gemini_Generate(gemini,setting)

In [8]:
def extract_json_dict(text):
    # Dùng regex để trích nội dung bên trong ```json ... ```
    match = re.search(r'```json\s*(\{.*?\})\s*```', text, re.DOTALL)
    if match:
        json_str = match.group(1)
        return json.loads(json_str)
    else:
        raise ValueError("Không tìm thấy nội dung JSON hợp lệ.")

In [9]:
result=gemini_func.extract_entities("Bao nhiêu tuổi thì phải đăng kí kết hôn?")
print(extract_json_dict(result))

{}


In [7]:
import time
from tqdm import tqdm
import pandas as pd
import json

def process_extract_entities(input_path, output_path, batch_size=10):
    # Đọc dữ liệu từ file Excel
    df = pd.read_excel(input_path, engine="openpyxl")
    queries = df["query"].tolist()
    labels = df["entities_label"].tolist()

    results = []

    for idx, (query, label_str) in enumerate(tqdm(zip(queries, labels), total=len(queries), desc="Extracting entities")):
        try:
            # Gọi Gemini để trích xuất entities
            raw_result = gemini_func.extract_entities(query)
            entity_dict = extract_json_dict(raw_result)
        except Exception as e:
            print(f"Lỗi ở dòng {idx + 1}: {e}")
            entity_dict = {}

        # Lưu kết quả (gồm cả nhãn thực tế)
        results.append({
            "query": query,
            "predict_entities": json.dumps(entity_dict, ensure_ascii=False),
            "entities_label": label_str,
        })

        # Ghi từng batch vào file
        if len(results) >= batch_size:
            _append_results_to_excel(results, output_path)
            results = []

        time.sleep(3)  # Tránh spam API

    # Ghi phần còn lại
    if results:
        _append_results_to_excel(results, output_path)

def _append_results_to_excel(results, output_path):
    result_df = pd.DataFrame(results)
    try:
        existing_df = pd.read_excel(output_path, engine="openpyxl")
        result_df = pd.concat([existing_df, result_df], ignore_index=True)
    except FileNotFoundError:
        pass
    result_df.to_excel(output_path, index=False, engine="openpyxl")
    print(f"Đã lưu {len(results)} kết quả vào {output_path}")


In [8]:
input_file = "../data_danhgia_module_router_generate/dataset_Query_Entities.xlsx"
output_file = "../data_danhgia_module_router_generate/eval_entities_extract.xlsx"

process_extract_entities(input_file, output_file)


Extracting entities:   1%|          | 9/906 [00:38<1:05:10,  4.36s/it]

Đã lưu 10 kết quả vào ../data_danhgia_module_router_generate/eval_entities_extract.xlsx


Extracting entities:   2%|▏         | 19/906 [01:20<1:01:20,  4.15s/it]

Đã lưu 10 kết quả vào ../data_danhgia_module_router_generate/eval_entities_extract.xlsx


Extracting entities:   3%|▎         | 29/906 [02:01<59:35,  4.08s/it]  

...

Đã lưu 10 kết quả vào ../data_danhgia_module_router_generate/eval_entities_extract.xlsx


Extracting entities:  99%|█████████▉| 899/906 [1:08:16<00:32,  4.62s/it]

Đã lưu 10 kết quả vào ../data_danhgia_module_router_generate/eval_entities_extract.xlsx


Extracting entities: 100%|██████████| 906/906 [1:08:51<00:00,  4.56s/it]


Đã lưu 6 kết quả vào ../data_danhgia_module_router_generate/eval_entities_extract.xlsx
